## Working with making schemas from JSONs

In [1]:
import os
import sys

notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
print(project_root)
sys.path.insert(0, project_root)

from src.DataCreator.DataSets.DataSetGenStandard import DataSetGenStandard
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("Test Data Set Generation") \
    .enableHiveSupport() \
    .master("local[*]") \
    .getOrCreate()

c:\Users\dalej\Documents\_Coding\DragonRegen


In [2]:
import pyodbc
driver = "ODBC Driver 17 for SQL Server"
server = 'localhost\\SQLEXPRESS' 
# server = "Andrew=PC\\SQLEXPRESS"
database = "MetaFort"
conn_str = (
    f"DRIVER={driver};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"Trusted_Connection=yes;"
)
connection = pyodbc.connect(conn_str)
cursor = connection.cursor()
print(f"Connected to MS SQL Server database: {database}")

Connected to MS SQL Server database: MetaFort


In [3]:
from src.DataCreator.SchemaGenerators.SchemaMSSQL import SchemaMSSQL
from src.MetaFort.AILoggingTables import AILoggingTables

s_sql_col = SchemaMSSQL.generate_schema_sql(AILoggingTables.d_system_tables)
print(s_sql_col)


{'actions': '    [action_id] INT NOT NULL,\n    [name] NVARCHAR(255) NOT NULL,\n    [description] NVARCHAR(255) NULL,\n    [created_at] DATETIME2 NOT NULL,\n    [updated_at] DATETIME2 NOT NULL,\n    [status] NVARCHAR(255) NOT NULL,\n    [error_code] NVARCHAR(255) NULL,\n    [error_message] NVARCHAR(255) NULL,\n    [error_timestamp] DATETIME2 NULL,\n    [metadata] NVARCHAR(MAX) NULL,\n    CONSTRAINT [PK_actions] PRIMARY KEY CLUSTERED ([action_id])', 'sub_actions': '    [sub_action_id] INT NOT NULL,\n    [action_id] INT NOT NULL,\n    [name] NVARCHAR(255) NOT NULL,\n    [description] NVARCHAR(255) NULL,\n    [sequence_number] INT NOT NULL,\n    [created_at] DATETIME2 NOT NULL,\n    [updated_at] DATETIME2 NOT NULL,\n    [status] NVARCHAR(255) NOT NULL,\n    [error_code] NVARCHAR(255) NULL,\n    [error_message] NVARCHAR(255) NULL,\n    [error_timestamp] DATETIME2 NULL,\n    [metadata] NVARCHAR(MAX) NULL,\n    CONSTRAINT [PK_sub_actions] PRIMARY KEY CLUSTERED ([sub_action_id])', 'requests':

In [4]:
output = SchemaMSSQL.create_tables_from_dict(db_engine=cursor, d_tables=s_sql_col)


        if OBJECT_ID('dbo.actions', 'U') IS NULL
        CREATE TABLE dbo.actions (    [action_id] INT NOT NULL,
    [name] NVARCHAR(255) NOT NULL,
    [description] NVARCHAR(255) NULL,
    [created_at] DATETIME2 NOT NULL,
    [updated_at] DATETIME2 NOT NULL,
    [status] NVARCHAR(255) NOT NULL,
    [error_code] NVARCHAR(255) NULL,
    [error_message] NVARCHAR(255) NULL,
    [error_timestamp] DATETIME2 NULL,
    [metadata] NVARCHAR(MAX) NULL,
    CONSTRAINT [PK_actions] PRIMARY KEY CLUSTERED ([action_id]))
        ;

        if OBJECT_ID('dbo.sub_actions', 'U') IS NULL
        CREATE TABLE dbo.sub_actions (    [sub_action_id] INT NOT NULL,
    [action_id] INT NOT NULL,
    [name] NVARCHAR(255) NOT NULL,
    [description] NVARCHAR(255) NULL,
    [sequence_number] INT NOT NULL,
    [created_at] DATETIME2 NOT NULL,
    [updated_at] DATETIME2 NOT NULL,
    [status] NVARCHAR(255) NOT NULL,
    [error_code] NVARCHAR(255) NULL,
    [error_message] NVARCHAR(255) NULL,
    [error_timestamp] DA

### Close Connection when done

In [5]:
cursor.close()
connection.close()